In [154]:
import music21
import pickle
import numpy as np
from music21 import duration

In [155]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Load data from files

https://verovio.humdrum.org/?file=essen/europa/deutschl/altdeu2/deut4207.krn

In [156]:
data = []

In [157]:
files = ["data_germany_1","data_germany_2","data_germany_3","data_germany_4","data_germany_5","data_germany_6","data_germany_7", "data_germany_8", "data_germany_9", "data_germany_10", "data_germany_11"]
for file in files:
    with open('../data/data_2/{}.pkl'.format(file), 'rb') as handle:
        data.extend(pickle.load(handle))

### Extracting the data

In [158]:
notes_input = []
durations_input = []
phrases_input = []

for piece in data:
    notes = piece[0]
    notes_input.append(129) # START = 129
    notes_input.extend(notes)

    durations = piece[1]
    durations_input.append(0)
    durations_input.extend(durations)

    phrases = piece[2]
    phrases_input.append(0)
    phrases_input.extend(phrases)

In [159]:
print("Length of the notes_input: {}".format(len(notes_input)))
print("Twenty first elements: {}".format(notes_input[:20]))
print("Length of the durations_input: {}".format(len(durations_input)))
print("Twenty first elements: {}".format(durations_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_input)))
print("Twenty first elements: {}".format(phrases_input[:20]))

Length of the notes_input: 282505
Twenty first elements: [129, 55, 60, 60, 60, 60, 60, 62, 64, 64, 64, 67, 62, 128, 67, 60, 128, 55, 60, 60]
Length of the durations_input: 282505
Twenty first elements: [0, 0.5, 0.5, 0.5, 0.375, 0.125, 0.25, 0.25, 0.5, 0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5, 0.5]
Length of the phrases_input: 282505
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [160]:
durations_to_int = {}
int_to_duration = {}
notes_to_int = {}
int_to_notes = {}
phrases_to_int = {}
int_to_phrases = {}
for index, duration in enumerate(sorted(set(durations_input))):
    durations_to_int[duration] = index
    int_to_duration[index] = duration

for index, note in enumerate(sorted(set(notes_input))):
    notes_to_int[note] = index
    int_to_notes[index] = note

for index, phrase in enumerate(sorted(set(phrases_input))):
    phrases_to_int[phrase] = index
    int_to_phrases[index] = phrase

durations_network_input = [ durations_to_int[duration] for duration in durations_input ]
notes_network_input = [ notes_to_int[note] for note in notes_input ]
phrases_network_input = [ phrases_to_int[phrase] for phrase in phrases_input ]

In [161]:
int_to_duration

{0: 0,
 1: 0.00390625,
 2: 0.125,
 3: Fraction(1, 6),
 4: 0.25,
 5: Fraction(1, 3),
 6: 0.375,
 7: 0.5,
 8: Fraction(2, 3),
 9: 0.75,
 10: 1.0,
 11: Fraction(4, 3),
 12: 1.5,
 13: 2.0,
 14: 3.0,
 15: 4.0,
 16: 6.0,
 17: 8.0,
 18: 12.0}

In [162]:
print("Length of the notes_input: {}".format(len(notes_network_input)))
print("Twenty first elements: {}".format(notes_network_input[:20]))
print("Length of the durations_input: {}".format(len(durations_network_input)))
print("Twenty first elements: {}".format(durations_network_input[:20]))
print("Length of the phrases_input: {}".format(len(phrases_network_input)))
print("Twenty first elements: {}".format(phrases_network_input[:20]))

Length of the notes_input: 282505
Twenty first elements: [47, 16, 21, 21, 21, 21, 21, 23, 25, 25, 25, 28, 23, 46, 28, 21, 46, 16, 21, 21]
Length of the durations_input: 282505
Twenty first elements: [0, 7, 7, 7, 6, 2, 4, 4, 7, 7, 7, 7, 10, 7, 7, 10, 7, 7, 7, 7]
Length of the phrases_input: 282505
Twenty first elements: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]


In [163]:
n_notes = len(notes_to_int)
n_durations = len(durations_to_int)
n_phrases = len(phrases_to_int)

In [164]:
print("Number of distinct notes: {}".format(n_notes))
print("Number of distinct durations: {}".format(n_durations))

Number of distinct notes: 48
Number of distinct durations: 19


In [165]:
with open('dictionary.pkl', 'wb') as file:
    dictionary = { "durations_to_int" : durations_to_int, "int_to_duration" : int_to_duration, "notes_to_int" : notes_to_int, "int_to_notes" : int_to_notes, "phrases_to_int" : phrases_to_int, "int_to_phrases" : int_to_phrases }
    # A new file will be created
    pickle.dump(dictionary, file)

In [166]:
import os
import pickle
import numpy
from music21 import note, chord

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model
from RNNmodel import prepare_sequences, create_network

In [167]:
seq_len = 5
network_input, network_output = prepare_sequences(notes_network_input, durations_network_input, phrases_network_input, seq_len)

In [168]:
print('note input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('phrases_output')
print(network_output[0][0])

note input
[47 16 21 21 21]
duration input
[0 7 7 7 6]
phrases_output
[1. 0.]


In [183]:
len(phrases_network_input)

282505

In [184]:
phrases_network_input

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,


In [181]:
network_input[0]

array([[47, 16, 21, 21, 21],
       [16, 21, 21, 21, 21],
       [21, 21, 21, 21, 21],
       ...,
       [21, 23, 24, 23, 21],
       [23, 24, 23, 21, 24],
       [24, 23, 21, 24, 23]])

In [187]:
def count_phrases(phrases_network_input):
    phrase_lengths = {}
    current_length = 0

    for digit in phrases_network_input:
        if digit == 0:
            current_length += 1
        else:
            if current_length > 0:
                if current_length not in phrase_lengths:
                    phrase_lengths[current_length] = 1
                else:
                    phrase_lengths[current_length] += 1
                current_length = 0

    if current_length > 0:
        if current_length not in phrase_lengths:
            phrase_lengths[current_length] = 1
        else:
            phrase_lengths[current_length] += 1

    sorted_phrase_lengths = dict(sorted(phrase_lengths.items()))

    return sorted_phrase_lengths


In [188]:
phrase_lengths = count_phrases(phrases_network_input)
print(phrase_lengths)

{1: 5, 2: 52, 3: 254, 4: 765, 5: 2342, 6: 5281, 7: 6355, 8: 5506, 9: 4146, 10: 2855, 11: 1889, 12: 910, 13: 546, 14: 287, 15: 135, 16: 85, 17: 56, 18: 33, 19: 20, 20: 16, 21: 7, 22: 6, 23: 9, 24: 1, 25: 2, 26: 1, 27: 1, 28: 2, 29: 1, 31: 1, 36: 1}


In [194]:
# keep phrases with lengths 4 - 12
def select_phrases(phrases_network_input, lower_bound=4, upper_bound=12):
    phrase_indices = []
    current_length = 0

    for i, digit in enumerate(phrases_network_input):
        if digit == 0:
            current_length += 1
        else:
            if current_length >= lower_bound and current_length <= upper_bound:
                phrase_indices.append(i - current_length)
            current_length = 0

    if current_length >= lower_bound and current_length <= upper_bound:
        phrase_indices.append(len(phrases_network_input) - current_length)

    new_phrases_network_input = []
    current_index = 0
    for i in phrase_indices:
        new_phrases_network_input.extend(phrases_network_input[current_index:i])
        current_index = i + (i in phrase_indices) * (upper_bound - lower_bound + 1) + 1
    new_phrases_network_input.extend(phrases_network_input[current_index:])

    return new_phrases_network_input

In [195]:
new_phrases_network_input = select_phrases(phrases_network_input, lower_bound=3, upper_bound=10)
print(new_phrases_network_input)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 

In [196]:
len(new_phrases_network_input)

61598

In [169]:
embed_size = 100
rnn_units = 256
use_attention = True
model, att_model = create_network(n_notes, n_durations, n_phrases, embed_size, rnn_units, use_attention)
model.summary()

Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_19 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 input_20 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_18 (Embedding)       (None, None, 100)    4800        ['input_19[0][0]']               
                                                                                                  
 embedding_19 (Embedding)       (None, None, 100)    1900        ['input_20[0][0]']               
                                                                                           

In [170]:
import numpy as np
weights_folder = "weights" #os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))


checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=20, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )

Epoch 1/20
7063/7063 [==============================] - 131s 18ms/step - loss: 0.2993 - val_loss: 0.2889
Epoch 2/20
7063/7063 [==============================] - 132s 19ms/step - loss: 0.2835 - val_loss: 0.2849
Epoch 3/20
7063/7063 [==============================] - 131s 19ms/step - loss: 0.2782 - val_loss: 0.2779
Epoch 4/20
7063/7063 [==============================] - 135s 19ms/step - loss: 0.2738 - val_loss: 0.2774
Epoch 5/20
7063/7063 [==============================] - 136s 19ms/step - loss: 0.2699 - val_loss: 0.2784
Epoch 6/20
7063/7063 [==============================] - 133s 19ms/step - loss: 0.2652 - val_loss: 0.2764
Epoch 7/20
7063/7063 [==============================] - 132s 19ms/step - loss: 0.2621 - val_loss: 0.2877
Epoch 8/20
7063/7063 [==============================] - 130s 18ms/step - loss: 0.2590 - val_loss: 0.2819
Epoch 9/20
7063/7063 [==============================] - 137s 19ms/step - loss: 0.2568 - val_loss: 0.2903
Epoch 10/20
7063/7063 [==============================] 